In [176]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


openjdk-8-jdk-headless is already the newest version (8u432-ga~us1-0ubuntu2~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [177]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

**Pipeline:** By properly splitting the input strings, one can obtain a complete list of mutual friends, and then by using self-join, groupby, sorting, and adding back users without secondary friends, one can get the desired outputs.

In [178]:
txt = spark.read.text("soc-LiveJournal1Adj.txt")

In [179]:
txt.take(3)

[Row(value='0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(value='1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(value='2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878')]

In [180]:
txt = txt.select(split(txt.value, "\t").alias('pair'))

In [181]:
txt.take(3)

[Row(pair=['0', '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94']),
 Row(pair=['1', '0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592']),
 Row(pair=['2', '0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'])]

In [182]:
txt = txt.select(txt.pair[0].alias('user'), txt.pair[1].alias('friends'))

In [183]:
txt.take(3)

[Row(user='0', friends='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(user='1', friends='0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(user='2', friends='0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878')]

In [184]:
txt = txt.select(txt.user, split(txt.friends, ",").alias('friends'))

In [185]:
friend_list = txt.select(txt.user, explode(txt.friends).alias('friend'))

In [186]:
friend_list.take(3)

[Row(user='0', friend='1'),
 Row(user='0', friend='2'),
 Row(user='0', friend='3')]

In [187]:
common_friend_list = friend_list.withColumnRenamed('user', 'user1').join(friend_list.withColumnRenamed('user', 'user2'), ['friend', 'friend'])

In [188]:
common_friend_list_count = common_friend_list.where(common_friend_list.user1 != common_friend_list.user2).groupBy(['user1', 'user2']).agg(count('*').alias('mutual_friends'))

In [189]:
common_friend_list_count.where("user1 == '11' and user2 == '27552'").show()

+-----+-----+--------------+
|user1|user2|mutual_friends|
+-----+-----+--------------+
|   11|27552|             4|
+-----+-----+--------------+



In [190]:
common_friend_list_count = common_friend_list_count.alias('a').join(friend_list.alias('b'), [col('a.user1') == col('b.user'), col('a.user2') == col('b.friend')], 'left_anti')

In [191]:
common_friend_list_count.where("user1 == '11' and user2 == '27552'").show()

+-----+-----+--------------+
|user1|user2|mutual_friends|
+-----+-----+--------------+
|   11|27552|             4|
+-----+-----+--------------+



In [192]:
common_friend_list_count = common_friend_list_count.select(common_friend_list_count.user1, common_friend_list_count.user2.cast("int"), (-1 * common_friend_list_count.mutual_friends).alias('mutual_friends'))

In [193]:
common_friend_list_count.where("user1 == '11' and user2 == 27552").show()

+-----+-----+--------------+
|user1|user2|mutual_friends|
+-----+-----+--------------+
|   11|27552|            -4|
+-----+-----+--------------+



In [194]:
common_friend_list_count = common_friend_list_count\
.groupBy('user1')\
.agg(sort_array(collect_list(struct('mutual_friends', 'user2'))).alias('sorted_pairs'))\
.withColumn('secondary_friends', col('sorted_pairs.user2')).drop('sorted_pairs')

In [195]:
output = common_friend_list_count.select('user1', slice(common_friend_list_count.secondary_friends, 1, 10).alias('secondary_friends')).withColumn('res', concat_ws(",", 'secondary_friends')).drop('secondary_friends')

In [196]:
output[output.user1.isin([924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993])].select(concat_ws("\t", 'user1', 'res').alias('res')).take(10)

[Row(res='9993\t9991,13134,13478,13877,34299,34485,34642,37941'),
 Row(res='9019\t9022,317,9023'),
 Row(res='924\t439,2409,6995,11860,15416,43748,45881'),
 Row(res='8942\t8939,8940,8943,8944'),
 Row(res='9020\t9021,9016,9017,9022,317,9023'),
 Row(res='9021\t9020,9016,9017,9022,317,9023'),
 Row(res='8941\t8943,8944,8940'),
 Row(res='9992\t9987,9989,35667,9991'),
 Row(res='9990\t13134,13478,13877,34299,34485,34642,37941'),
 Row(res='9022\t9019,9020,9021,317,9016,9017,9023')]

In [197]:
final_results = txt.join(output, txt.user == output.user1, "left").na.fill({'res' : ""}).select( concat_ws("\t", 'user', 'res').alias('res'))

*Take a look at users without secondary friends.*

In [198]:
temp = txt.join(output, txt.user == output.user1, "left")
temp[temp.user1.isNull()].show()

+-----+--------------------+-----+----+
| user|             friends|user1| res|
+-----+--------------------+-----+----+
|10365|             [10415]| NULL|NULL|
|10372|             [10440]| NULL|NULL|
|10440|             [10372]| NULL|NULL|
|11830|      [11831, 11832]| NULL|NULL|
| 1473|              [1474]| NULL|NULL|
|15016|             [15015]| NULL|NULL|
|15077|             [15078]| NULL|NULL|
|15078|             [15077]| NULL|NULL|
|18077|      [18078, 18079]| NULL|NULL|
|18891|[18892, 18893, 18...| NULL|NULL|
|18895|[18891, 18893, 18...| NULL|NULL|
|18902|[18891, 18893, 18...| NULL|NULL|
|22455|             [22454]| NULL|NULL|
|22897|[22898, 22899, 22...| NULL|NULL|
|23057|             [23021]| NULL|NULL|
|24332|             [24333]| NULL|NULL|
|24333|             [24332]| NULL|NULL|
|24934|             [24857]| NULL|NULL|
|25432|             [25431]| NULL|NULL|
|29424|             [29423]| NULL|NULL|
+-----+--------------------+-----+----+
only showing top 20 rows



In [199]:
final_results.where(final_results.res.startswith("10365")).take(1)

[Row(res='10365\t')]

In [200]:
final_results.take(5)

[Row(res='0\t38737,18591,27383,34211,337,352,1532,12143,12561,17880'),
 Row(res='3\t27679,1,10,16,29,30,38,82,83,85'),
 Row(res='1\t35621,44891,14150,15356,35630,13801,13889,14078,25228,13805'),
 Row(res='2\t41087,1,5,95,112,1085,1404,2411,3233,4875'),
 Row(res='4\t13,16,19,30,38,46,83,85,89,125')]